In [0]:
# https://reurl.cc/VWZjQ

In [0]:
from google.colab import drive
drive.mount('/gdrive')

In [0]:
import zipfile
f = zipfile.ZipFile("/gdrive/My Drive/trans.zip")
f.extractall("news")
f = zipfile.ZipFile("/gdrive/My Drive/test.zip")
f.extractall("newstest")

In [0]:
import glob
import pandas as pd
traindf = pd.DataFrame(columns=["content", "category"])
for fn in glob.glob("news/chinese_news_trans/*/*.txt"):
    f = open(fn, "r", encoding="utf-8")
    article = f.read()
    f.close()
    cat = fn.split("/")[-2]
    s = pd.Series([article, cat], index=["content", "category"])
    traindf = traindf.append(s, ignore_index=True)
for fn in glob.glob("news/chinese_news_trans/*/*.TXT"):
    f = open(fn, "r", encoding="utf-8")
    article = f.read()
    f.close()
    cat = fn.split("/")[-2]
    s = pd.Series([article, cat], index=["content", "category"])
    traindf = traindf.append(s, ignore_index=True)
traindf

In [0]:
testdf = pd.DataFrame(columns=["content", "category"])
for fn in glob.glob("newstest/chinese_news_test/*/*.txt"):
    f = open(fn, "r", encoding="utf-8")
    article = f.read()
    f.close()
    cat = fn.split("/")[-2]
    s = pd.Series([article, cat], index=["content", "category"])
    testdf = testdf.append(s, ignore_index=True)
for fn in glob.glob("newstest/chinese_news_test/*/*.TXT"):
    f = open(fn, "r", encoding="utf-8")
    article = f.read()
    f.close()
    cat = fn.split("/")[-2]
    s = pd.Series([article, cat], index=["content", "category"])
    testdf = testdf.append(s, ignore_index=True)
testdf

In [0]:
import jieba
def cutflow(s):
    return " ".join(jieba.cut(s)).replace("\r", "").replace("\n", "")
traindf["content"] = traindf["content"].apply(cutflow)
testdf["content"] = testdf["content"].apply(cutflow)
testdf

In [0]:
cat = traindf["category"].unique()
# {"環境":0}
trans = {c:i for (i, c) in enumerate(cat)}
traindf["category"] = traindf["category"].replace(trans)
testdf["category"] = testdf["category"].replace(trans)
testdf

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train = vec.fit_transform(traindf["content"])
x_test = vec.transform(testdf["content"])

In [41]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train, traindf["category"])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [42]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test)
accuracy_score(pre, testdf["category"])

1.0